<a href="https://colab.research.google.com/github/TF2113/mobility-aid/blob/master/augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Augumentation using Albumentations

Apply data augmentation to a directory of images using Albumentations library for Python. A tool for increasing the amount of viable data for training machine learning models on a small initial dataset.

#Required Libraries and Packages Installer

Code snippet to install all requirements for Albumentations to run correctly.

In [ ]:
# Install required packages quietly
!pip install albumentations opencv-python matplotlib --quiet

# Imports
import os
import cv2
import albumentations as A
from matplotlib import pyplot as plt
from google.colab import drive

# Mount Google Drive to access files
drive.mount('/content/drive')

print("Setup complete! You can now access your images and use Albumentations.")


Mounted at /content/drive
Setup complete! You can now access your images and use Albumentations.


# Set Input and Output Directories

Upload data.zip containing labelled images in YOLO format to be unzipped. Made using Label Studio for this project.

In [ ]:
import zipfile

zip_path = '/content/drive/MyDrive/data.zip'   # Path to zip file containing images and labels
extract_to = '/content/drive/MyDrive/'         # Path to extract data to

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Data extracted to: {extract_to}")

Data extracted to: /content/drive/MyDrive/


Set path to input/output directories for storing augmented images.

In [ ]:
input_img_dir = '/content/drive/MyDrive/images'         # Your images folder
input_label_dir = '/content/drive/MyDrive/labels'       # Your YOLO txt labels folder

output_img_dir = '/content/drive/MyDrive/aug_images'    # Where augmented images go
output_label_dir = '/content/drive/MyDrive/aug_labels'  # Where augmented labels go

os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

print(f"Input images: {input_img_dir}")
print(f"Input labels: {input_label_dir}")
print(f"Output images: {output_img_dir}")
print(f"Output labels: {output_label_dir}")


Input images: /content/drive/MyDrive/images
Input labels: /content/drive/MyDrive/labels
Output images: /content/drive/MyDrive/aug_images
Output labels: /content/drive/MyDrive/aug_labels


#Helper Functions for Reading/Writing Labels

Code to define two functions that will read and write YOLO label files when needed for Albumentations augmentations.

In [ ]:
def read_yolo_labels(label_file_path):
  bounding_boxes = []
  class_ids = []
  with open(label_file_path, 'r') as label_file: # Open .txt file for image label
    for line in label_file:
      sections = line.strip().split()            # Split each line into sections
      class_id = int(sections[0])                # YOLO label format begins with class ids
      bounding_box = []
      for part in sections[1:5]:                 # Next section in YOLO format is the x,y of the bounding boxes
        bounding_box.append(float(part))         # alongside the height and width of the box
      class_ids.append(class_id)
      bounding_boxes.append(bounding_box)
  return bounding_boxes, class_ids               # Return arrays containing locations for all bounding boxes and labels in the image

def save_yolo_labels(label_path, bounding_boxes, class_ids):
    with open(label_path, 'w') as label_file:                               # Open file to save to
        for class_id, bounding_box in zip(class_ids, bounding_boxes):
            formatted_coords = []
            for coord in bounding_box:
                formatted_coord = format(coord, ".6f")                      # Format coords to 6 decimal places
                formatted_coords.append(formatted_coord)

            bounding_box_line = f"{class_id} " + ' '.join(formatted_coords) # Saves formatted line to the file (Formatted example = 1 0.123456 0.123456 0.123456 0.123456)
            label_file.write(bounding_box_line + "\n")




#Augmentation Pipeline

Defining the image augmentations to be used and their probabilities of being applied for each image. Augmentations are applied randomly to improve diversity of the data and help avoid overfitting to similar images.

In [ ]:
pipeline = A.Compose([
    A.RandomSizedBBoxSafeCrop(height = 640, width = 640, p=0.75),
    A.HorizontalFlip(p=0.6),
    A.SafeRotate(limit=[-10, 10], p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.RandomGamma(p=0.4),
    A.GaussianBlur(p=0.3),
    A.GaussNoise(p=0.15),
    A.RandomRain(p=0.2),
    A.RandomFog(p=0.05),
    A.RandomSunFlare(p=0.05),
    A.Resize(height = 640, width = 640)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_ids']))

# Augmenting the Images

Applying the defined pipeline to each image in the */image* directory 5 times to provide a diverse dataset of ~300 images from the initial dataset of 61 images, outputting to the */aug_images* directory.

In [ ]:
for file in os.listdir(input_img_dir):
  if not file.lower().endswith(".jpeg"):
    continue

  img_path = os.path.join(input_img_dir, file)
  label_path = os.path.join(input_label_dir, file.replace(".jpeg", ".txt"))

  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  bounding_boxes, class_ids = read_yolo_labels(label_path)

  for i in range(5):
    augmented_image = pipeline(image=img, bboxes=bounding_boxes, class_ids=class_ids)
    aug_img = augmented_image['image']
    aug_bounding_boxes = augmented_image['bboxes']
    aug_class_ids = augmented_image['class_ids']

    file_name = os.path.splitext(file)[0]
    aug_img_bgr = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)
    aug_image_path = os.path.join(output_img_dir, f"{file_name}_aug{i}.jpeg")
    aug_label_path = os.path.join(output_label_dir, f"{file_name}_aug{i}.txt")

    cv2.imwrite(aug_image_path, aug_img_bgr)
    save_yolo_labels(aug_label_path, aug_bounding_boxes, aug_class_ids)

Zip all image and label data to be used in training the model.

In [ ]:
!zip -r data.zip /content/drive/MyDrive/aug_images /content/drive/MyDrive/aug_labels /content/drive/MyDrive/images /content/drive/MyDrive/labels



  adding: content/drive/MyDrive/aug_images/ (stored 0%)
  adding: content/drive/MyDrive/aug_images/1e8f0528-WhatsApp_Image_2025-06-11_at_23.32.45.jpeg (deflated 0%)
  adding: content/drive/MyDrive/aug_images/0d41a1a1-WhatsApp_Image_2025-06-13_at_20.15.54.jpeg (deflated 0%)
  adding: content/drive/MyDrive/aug_images/0aca2422-WhatsApp_Image_2025-06-11_at_23.32.48_2.jpeg (deflated 0%)
  adding: content/drive/MyDrive/aug_images/21d15edb-WhatsApp_Image_2025-06-11_at_23.32.43_1.jpeg (deflated 0%)
  adding: content/drive/MyDrive/aug_images/0e561c9e-WhatsApp_Image_2025-06-11_at_23.32.47_1.jpeg (deflated 0%)
  adding: content/drive/MyDrive/aug_images/1d3c5b22-WhatsApp_Image_2025-06-13_at_20.15.56.jpeg (deflated 1%)
  adding: content/drive/MyDrive/aug_images/48870751-WhatsApp_Image_2025-06-14_at_01.56.58.jpeg (deflated 0%)
  adding: content/drive/MyDrive/aug_images/34aa4dac-WhatsApp_Image_2025-06-11_at_23.32.48.jpeg (deflated 0%)
  adding: content/drive/MyDrive/aug_images/40740376-WhatsApp_Image